<center><h1> Fraud Shopee </h1></center>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Chuẩn bị

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

In [3]:
class StringConverter(dict):
    def __contains__(self, item):
        return True

    def __getitem__(self, item):
        return str

    def get(self, default=None):
        return str

In [4]:
bank_accounts = pd.read_csv("/content/drive/Othercomputers/My Laptop/Shopee/fraud_shopee/bank_accounts.csv", converters=StringConverter())
credit_card = pd.read_csv("/content/drive/Othercomputers/My Laptop/Shopee/fraud_shopee/credit_cards.csv", converters=StringConverter())
devices = pd.read_csv("/content/drive/Othercomputers/My Laptop/Shopee/fraud_shopee/devices.csv", converters=StringConverter())
orders = pd.read_csv("/content/drive/Othercomputers/My Laptop/Shopee/fraud_shopee/orders.csv", converters=StringConverter())
submiss = pd.read_csv("/content/drive/Othercomputers/My Laptop/Shopee/fraud_shopee/submission.csv", converters=StringConverter())

In [5]:
# bank_accounts = pd.read_csv("fraud_shopee\\bank_accounts.csv", converters=StringConverter())
# credit_card = pd.read_csv("fraud_shopee\\credit_cards.csv", converters=StringConverter())
# devices = pd.read_csv("fraud_shopee\\devices.csv", converters=StringConverter())
# orders = pd.read_csv("fraud_shopee\\orders.csv", converters=StringConverter())
# submiss = pd.read_csv("fraud_shopee\submission.csv", converters=StringConverter())

-------------------------

# <center><h4>Gian lận trực tiếp</h4></center>

## Khắc phục các vấn đề

* Khắc phục các vấn đề:
    * Bảng orderid, chúng ta chỉ xem xét thằng buyer, bởi vì.
        * Nếu người bán mà có trong người mua --> Có vấn đề.
        * Và người bán mà không có trong người mua --> Không có vấn đề
    * Kiểm tra thiết bị của buyer
        * Nếu số lượng thiết bị lớn hơn 1 ....
        * Nếu số lượng thiết bị nhỏ hơn 1 ....

## Bảng orders

- Bảng orderid, chúng ta chỉ xem xét thằng buyer, bởi vì.
    + Người bán mà có trong người mua --> Có vấn đề.
    + Người bán mà không có trong người mua --> Không có vấn đề
- Vì vậy, ta sẽ chỉ xét người mua

In [6]:
temp = orders

In [7]:
temp = pd.DataFrame(temp)

In [8]:
temp = temp.reset_index()

In [9]:
temp

,index,orderid,buyer_userid,seller_userid
0,0,1953278092,47388162,20822974
1,1,1953295120,26855196,16416890
2,2,1953306402,121296714,28477978
3,3,1953314712,131221930,72837912
4,4,1953381964,183398314,28423332
...,...,...,...,...
620942,620942,1956636054,14222102,70940136
620943,620943,1956642760,2978154,60766142
620944,620944,1956651496,221974772,15541938
620945,620945,1956660812,152533590,80986700


In [10]:
temp = temp.drop('index', axis=1)
temp

,orderid,buyer_userid,seller_userid
0,1953278092,47388162,20822974
1,1953295120,26855196,16416890
2,1953306402,121296714,28477978
3,1953314712,131221930,72837912
4,1953381964,183398314,28423332
...,...,...,...
620942,1956636054,14222102,70940136
620943,1956642760,2978154,60766142
620944,1956651496,221974772,15541938
620945,1956660812,152533590,80986700


So sánh với việc tìm 620947 giao dịch thì tìm trên 204842 đã nhanh hơn 2/3 thời gian.

## Bảng User có đơn hàng

In [11]:
buyer_userid_co_don_hang = pd.DataFrame(temp[['orderid','buyer_userid', 'seller_userid']])
buyer_userid_co_don_hang

,orderid,buyer_userid,seller_userid
0,1953278092,47388162,20822974
1,1953295120,26855196,16416890
2,1953306402,121296714,28477978
3,1953314712,131221930,72837912
4,1953381964,183398314,28423332
...,...,...,...
620942,1956636054,14222102,70940136
620943,1956642760,2978154,60766142
620944,1956651496,221974772,15541938
620945,1956660812,152533590,80986700


## Kiểm tra devices của những người mua

    * Ta sẽ join-left bảng *buyer_userid_co_don_hang* với bảng *devices*

In [12]:
buyer_userid_co_don_hang_device = buyer_userid_co_don_hang.merge(devices, how='left', left_on='buyer_userid', right_on='userid')
buyer_userid_co_don_hang_device

,orderid,buyer_userid,seller_userid,userid,device
0,1953278092,47388162,20822974,47388162,N6EVAPjl6WH2r1LOkkuwR0zKxT3fU8tubs3qvysuKTy=
1,1953278092,47388162,20822974,47388162,NUDUHm6TKnxRN2Q+D7Wr2AAR5IyNabr1t3tobdD8uyq=
2,1953278092,47388162,20822974,47388162,HJ5XUYvJpjF6xE1QlTAy0RQgXY1GZDskcoLSliOMNag=
3,1953278092,47388162,20822974,47388162,5YKQkZDv4zi4RICd0AIVAS3Iv+awDbNxl49p51QF7fA=
4,1953278092,47388162,20822974,47388162,HQgnByUSw54FHZwYDGgwCheMM72dnyBqqF7lDih1iMW=
...,...,...,...,...,...
1709743,1956679514,54374608,10385548,54374608,s9tldgil9NLlyzVIv2BvkPQJIfUDGlTi5sYN6wy5640=
1709744,1956679514,54374608,10385548,54374608,LQzl2WwY7aseGyS4rlgYDiWsfZowlF0LdHVSbJs/Am8=
1709745,1956679514,54374608,10385548,54374608,mgvylksLNwhX/zwvYcm3Dlxf4KUEGW6VLasAgVWh8h8=
1709746,1956679514,54374608,10385548,54374608,wBUFxlVwv3xaiM73yAywq9X2TV5DaC6+8YGgY2kJr4E=


    * Kiểm tra thiết bị của buyer
        * Nếu số lượng thiết bị lớn hơn 1 --> userid có thể có xảy ra gian lận
        * Nếu số lượng thiết bị nhỏ hơn 1 --> userid không xảy ra, vì không có giao dịch, không thể có devices chung

    * Lấy ra các đơn hàng orderid mà có buyer_userid không sử dụng thiết bị nào

In [13]:
don_hang_ma_buyer_userid_device = buyer_userid_co_don_hang_device.dropna()
don_hang_ma_buyer_userid_device

,orderid,buyer_userid,seller_userid,userid,device
0,1953278092,47388162,20822974,47388162,N6EVAPjl6WH2r1LOkkuwR0zKxT3fU8tubs3qvysuKTy=
1,1953278092,47388162,20822974,47388162,NUDUHm6TKnxRN2Q+D7Wr2AAR5IyNabr1t3tobdD8uyq=
2,1953278092,47388162,20822974,47388162,HJ5XUYvJpjF6xE1QlTAy0RQgXY1GZDskcoLSliOMNag=
3,1953278092,47388162,20822974,47388162,5YKQkZDv4zi4RICd0AIVAS3Iv+awDbNxl49p51QF7fA=
4,1953278092,47388162,20822974,47388162,HQgnByUSw54FHZwYDGgwCheMM72dnyBqqF7lDih1iMW=
...,...,...,...,...,...
1709743,1956679514,54374608,10385548,54374608,s9tldgil9NLlyzVIv2BvkPQJIfUDGlTi5sYN6wy5640=
1709744,1956679514,54374608,10385548,54374608,LQzl2WwY7aseGyS4rlgYDiWsfZowlF0LdHVSbJs/Am8=
1709745,1956679514,54374608,10385548,54374608,mgvylksLNwhX/zwvYcm3Dlxf4KUEGW6VLasAgVWh8h8=
1709746,1956679514,54374608,10385548,54374608,wBUFxlVwv3xaiM73yAywq9X2TV5DaC6+8YGgY2kJr4E=


    * Tìm ra các gian lận trực tiếp bằng cách join - left bảng trên với bảng devices bằng key 'device'

In [14]:
temp = don_hang_ma_buyer_userid_device.merge(devices, how='left', left_on='device', right_on='device')
fraud1 = temp[temp['seller_userid'] == temp['userid_y']]
fraud1

,orderid,buyer_userid,seller_userid,userid_x,device,userid_y
7673,1955598428,35545436,70763052,35545436,/3TLpeou8xXsNxpACFFKr34Kqqwxiu5Hi1keJ6plk5E=,70763052
10188,1954515646,32834366,188151804,32834366,1KNEOFRIZaFcFx5+S+bOxyWuWBbITxnfoM7j6aZg05y=,188151804
17164,1953728724,168491444,158559422,168491444,yf7AHm3O97XAQwQuSmyoaxcaFSSAZcVCxm5YNgiEo4S=,158559422
32436,1956165008,29689348,231810424,29689348,bGvR9cey+rry1FhUJrRw9djRmT9BtJlGu/vM8dbUQU0=,231810424
39826,1954558468,235452908,235460800,235452908,V8hLxnXTJ0ua3SyVjCt3D06uZ8sofMHU3NxVjS7k7Ay=,235460800
...,...,...,...,...,...,...
1780838,1953994554,30153396,129295230,30153396,U+J33IBG2WOcbS1jGj2GdsLQb7GEilLaKjHIL29U0b0=,129295230
1787689,1955929340,235579716,80629986,235579716,4frUznPkj1NuGyDLkPfovV3mHm1MSGxIrXaz68I4n4m=,80629986
1792923,1954626246,197784360,12132476,197784360,79NZWlJcqP6R4VsNMt0ZNnMfURvO4RNIMa5mjV/dbTm=,12132476
1797809,1953812320,235299524,229915488,235299524,QWPloLVNUPpaxlUjlJQytCXEc5FLy7HkzCcCX0DWdzg=,229915488


In [15]:
fraud1.groupby('orderid').count().count()

buyer_userid     291
seller_userid    291
userid_x         291
device           291
userid_y         291
dtype: int64

Có 291 giao dịch gian lận trực tiếp bằng devices

## Kiểm tra credit_cards của những người mua

    * Ta sẽ join với bảng *buyer_userid_co_don_hang* với bảng *credit_cards*

In [16]:
buyer_userid_co_don_hang_credit_cards = buyer_userid_co_don_hang.merge(credit_card, how='left', left_on='buyer_userid', right_on='userid')
buyer_userid_co_don_hang_credit_cards = buyer_userid_co_don_hang_credit_cards
buyer_userid_co_don_hang_credit_cards

,orderid,buyer_userid,seller_userid,userid,credit_card
0,1953278092,47388162,20822974,NaN,NaN
1,1953295120,26855196,16416890,NaN,NaN
2,1953306402,121296714,28477978,NaN,NaN
3,1953314712,131221930,72837912,NaN,NaN
4,1953381964,183398314,28423332,NaN,NaN
...,...,...,...,...,...
641169,1956636054,14222102,70940136,14222102,333008xxxxxx9209|05-2021
641170,1956642760,2978154,60766142,NaN,NaN
641171,1956651496,221974772,15541938,NaN,NaN
641172,1956660812,152533590,80986700,NaN,NaN


    * Tương tự như devices
        * Nếu số credit_card >=1 --> userid có thể có xảy ra gian lận
        * Nếu số credit_card < 1 --> userid không thể xảy ra gian lận, không có devices chung

In [17]:
# Các đơn hàng mà buyer_userid có ít nhất một thẻ thanh toán
orderid_ma_buyer_userid_credit_cards = buyer_userid_co_don_hang_credit_cards.dropna()
orderid_ma_buyer_userid_credit_cards

,orderid,buyer_userid,seller_userid,userid,credit_card
54,1953991902,38165584,86977176,38165584,830129xxxxxx3008|04-2020
78,1954202536,9589270,9516814,9589270,320808xxxxxx9410|04-2020
105,1954529620,2218588,183582984,2218588,830129xxxxxx4402|07-2020
121,1954722184,11192106,41059918,11192106,392843xxxxxx2880|09-2021
122,1954722184,11192106,41059918,11192106,333843xxxxxx1921|11-2021
...,...,...,...,...,...
641102,1955750968,26389652,155518232,26389652,390212xxxxxx0909|08-2020
641103,1955750968,26389652,155518232,26389652,322421xxxxxx2441|04-2018
641112,1955918172,27005976,63038144,27005976,830422xxxxxx9029|06-2019
641113,1955918172,27005976,63038144,27005976,393922xxxxxx9408|04-2022


    * Tìm các giao dịch gian lận trực tiếp bằng cách join - left bảng trên với bảng *credit_cards* bằng key 'credit_card'

In [18]:
temp = orderid_ma_buyer_userid_credit_cards.merge(credit_card, how='left', left_on='credit_card', right_on='credit_card')
fraud2 = temp[temp['seller_userid'] == temp['userid_y']]
fraud2
# temp

,orderid,buyer_userid,seller_userid,userid_x,credit_card,userid_y


Không có gian lận bằng thẻ thanh toán

## Kiểm tra bank_account của những người mua

    * Tương tự đối với bank_account
        * Ta sẽ Join bảng "buyer_userid_co_don_hang" với bảng "bank_account"
        * Loại bỏ những oderid mà buyer_userid không có thông tin tài khoản (số lượng tài khoản < 1)

In [19]:
buyer_userid_co_don_hang_bank_account = buyer_userid_co_don_hang.merge(bank_accounts, how='left', left_on='buyer_userid', right_on='userid')
buyer_userid_co_don_hang_bank_account1 = buyer_userid_co_don_hang_bank_account.dropna()
buyer_userid_co_don_hang_bank_account1

,orderid,buyer_userid,seller_userid,userid,bank_account
0,1953278092,47388162,20822974,47388162,2210020214382
3,1953314712,131221930,72837912,131221930,0244224839
6,1953451192,89510674,18186196,89510674,093302090192804
7,1953453596,198638326,7314602,198638326,2920333989
9,1953480060,130236220,77145726,130236220,2094902840023493
...,...,...,...,...,...
699096,1956636054,14222102,70940136,14222102,4930104411
699097,1956636054,14222102,70940136,14222102,4930292828
699098,1956642760,2978154,60766142,2978154,4442220120
699100,1956660812,152533590,80986700,152533590,9910293034


    * Tìm các giao dịch gian lận trực tiếp bằng cách join - left bảng trên với bank_account bằng key 'bank_account'

In [20]:
temp = buyer_userid_co_don_hang_bank_account1.merge(bank_accounts, how='left', left_on='bank_account', right_on='bank_account')
fraud3 = temp[temp['seller_userid'] == temp['userid_y']]
fraud3

,orderid,buyer_userid,seller_userid,userid_x,bank_account,userid_y
1027,1954198318,221232712,66353306,221232712,8300298809,66353306
5468,1955955178,235599454,51098362,235599454,9120282009,51098362
9026,1954692012,104954088,93237386,104954088,229002004223803,93237386
22033,1954398710,210580366,5349078,210580366,1000023914228,5349078
26059,1954169068,235427122,74415764,235427122,029302092091803,74415764
29570,1955252130,100918044,68742042,100918044,0390313892,68742042
33929,1953890506,4748430,21964656,4748430,2212934039,21964656
34813,1954307678,162360172,64897430,162360172,2020292031,64897430
38921,1956337482,174397422,172696026,174397422,9828092929,172696026
38922,1956337482,174397422,172696026,174397422,9828092929,172696026


In [21]:
xx = pd.DataFrame(fraud3)
xx.count()

orderid          57
buyer_userid     57
seller_userid    57
userid_x         57
bank_account     57
userid_y         57
dtype: int64

Có 57 giao dịch gian lận bằng bank_account

In [22]:
57 + 291

348

In [23]:
submiss[submiss['is_fraud'] == '1']

,orderid,is_fraud
1649,1954198318,1
2679,1955598428,1
3545,1954515646,1
5938,1953728724,1
8393,1955955178,1
...,...,...
612805,1955929340,1
614563,1954626246,1
616320,1953812320,1
616725,1954821958,1


In [24]:
478 - 348

130

-------------------------------------------------------------------------------------------

# <center><h4>Giao dịch gián tiếp</h4></center>

In [25]:
orders

,orderid,buyer_userid,seller_userid
0,1953278092,47388162,20822974
1,1953295120,26855196,16416890
2,1953306402,121296714,28477978
3,1953314712,131221930,72837912
4,1953381964,183398314,28423332
...,...,...,...
620942,1956636054,14222102,70940136
620943,1956642760,2978154,60766142
620944,1956651496,221974772,15541938
620945,1956660812,152533590,80986700


## Tiền xử lý bảng order thành new_orders. Loại bỏ tất các các orderid có buyer_userid không có thiết bị nào

In [26]:
new_orders = pd.merge(orders, devices, how='left', left_on = 'buyer_userid', right_on = 'userid')
# new_orders[new_orders.isna().any(axis=1)] #Kiểm tra NaN
new_orders = new_orders.dropna()
new_orders = new_orders.drop({'userid', 'device'}, axis=1)
new_orders = new_orders.drop_duplicates()
new_orders = new_orders.reset_index(drop=True)
new_orders

,orderid,buyer_userid,seller_userid
0,1953278092,47388162,20822974
1,1953295120,26855196,16416890
2,1953306402,121296714,28477978
3,1953314712,131221930,72837912
4,1953381964,183398314,28423332
...,...,...,...
619475,1956636054,14222102,70940136
619476,1956642760,2978154,60766142
619477,1956651496,221974772,15541938
619478,1956660812,152533590,80986700


## Tạo bảng dic chứa thông tin của user và device

In [27]:
dic = new_orders.merge(devices, how='left', left_on='buyer_userid', right_on='userid')
dic = dic[['buyer_userid', 'device']]
dic = dic.rename(columns={'buyer_userid' : 'userid'})
dic

,userid,device
0,47388162,N6EVAPjl6WH2r1LOkkuwR0zKxT3fU8tubs3qvysuKTy=
1,47388162,NUDUHm6TKnxRN2Q+D7Wr2AAR5IyNabr1t3tobdD8uyq=
2,47388162,HJ5XUYvJpjF6xE1QlTAy0RQgXY1GZDskcoLSliOMNag=
3,47388162,5YKQkZDv4zi4RICd0AIVAS3Iv+awDbNxl49p51QF7fA=
4,47388162,HQgnByUSw54FHZwYDGgwCheMM72dnyBqqF7lDih1iMW=
...,...,...
1708276,54374608,s9tldgil9NLlyzVIv2BvkPQJIfUDGlTi5sYN6wy5640=
1708277,54374608,LQzl2WwY7aseGyS4rlgYDiWsfZowlF0LdHVSbJs/Am8=
1708278,54374608,mgvylksLNwhX/zwvYcm3Dlxf4KUEGW6VLasAgVWh8h8=
1708279,54374608,wBUFxlVwv3xaiM73yAywq9X2TV5DaC6+8YGgY2kJr4E=



Hàm phát hiện gian lận gián tiếp qua thiết bị


In [28]:
def detection_by_device(new_orders, devices):
  new_devices = pd.merge(new_orders, devices, how='left', left_on='buyer_userid', right_on='userid').drop('userid', axis=1)

  new_userid_related = pd.merge(new_devices, devices, how='left', left_on='device', right_on='device').dropna()
  new_userid_related = new_userid_related.rename(columns={'userid':'userid_relate'})
  new_userid_related = new_userid_related[new_userid_related['buyer_userid'] != new_userid_related['userid_relate']] # Xóa buyer_userid == userid_relate
  new_userid_related = new_userid_related.drop('device', axis = 1)
  new_userid_related = new_userid_related.drop_duplicates()




In [29]:
# Mức 1 là thằng A và thằng B chung 1 chi tiết
# Mức 2 là thằng A chung thằng A' một chi tiết thằng A' chung B 1 chi tiết
# Mức 3 là thằng A chung thằng A' một chi tiết thằng A' chung với A'' 1 chi tiết A'' chung B 1 chi tiết


In [30]:

new_devices = pd.merge(new_orders, devices, how='left', left_on='buyer_userid', right_on='userid').drop('userid', axis=1)

new_userid_related = pd.merge(new_devices, devices, how='left', left_on='device', right_on='device').dropna()
if 'userid_relate' in new_userid_related.columns:
    new_userid_related = new_userid_related.drop('userid_relate', axis=1)
new_userid_related = new_userid_related.rename(columns={'userid':'userid_relate'})
new_userid_related = new_userid_related[new_userid_related['buyer_userid'] != new_userid_related['userid_relate']] # Xóa buyer_userid == userid_relate
new_userid_related = new_userid_related.drop('device', axis = 1) # Xóa cột devices
new_userid_related = new_userid_related.drop_duplicates() # Xóa duplicates
Fraud = new_userid_related[new_userid_related['seller_userid'] == new_userid_related['userid_relate']] #Lựa chọn những thằng gian lận
print(Fraud['orderid'])
fraud = pd.DataFrame(Fraud['orderid'])


for _ in range(2):
  new_userid = new_userid_related[new_userid_related['seller_userid'] != new_userid_related['userid_relate']] #Lọc những thằng đã gian lận ra ngoài
  new_userid
  new_devices = pd.merge(new_userid, dic, how='left', left_on='userid_relate', right_on='userid').drop('userid', axis=1)


  new_userid_related = pd.merge(new_devices, devices, how='left', left_on='device', right_on='device').dropna()
  new_userid_related = new_userid_related[new_userid_related['userid_relate'] != new_userid_related['userid']]
  if 'userid_relate' in new_userid_related.columns:
      new_userid_related = new_userid_related.drop('userid_relate', axis=1)
  new_userid_related = new_userid_related.rename(columns={'userid':'userid_relate'})
  new_userid_related = new_userid_related[new_userid_related['buyer_userid'] != new_userid_related['userid_relate']]
  new_userid_related = new_userid_related.drop('device', axis = 1)
  new_userid_related = new_userid_related.drop_duplicates()
  Fraud = new_userid_related[new_userid_related['seller_userid'] == new_userid_related['userid_relate']]
  print(Fraud['orderid'])
  fraud = pd.concat([fraud, pd.DataFrame(Fraud['orderid'])], axis = 0)

fraud


7673       1955598428
10188      1954515646
17164      1953728724
32436      1956165008
39826      1954558468
              ...    
1780838    1953994554
1787689    1955929340
1792923    1954626246
1797809    1953812320
1802800    1955615664
Name: orderid, Length: 291, dtype: object
12923     1953543830
22445     1954558468
29546     1955618372
47042     1955841786
56613     1954576356
             ...    
884791    1955832452
886189    1955626470
902851    1953539378
910453    1953556460
911857    1955655890
Name: orderid, Length: 83, dtype: object
5621      1953543830
16300     1955618372
26698     1955841786
56972     1953541694
74781     1955642868
75616     1953556948
95943     1953555124
137023    1953388498
138295    1953346572
139426    1953556004
144159    1955898808
155010    1955823850
159541    1953544576
160274    1954199440
174453    1955364262
188776    1953543162
201781    1955212714
218320    1955778924
220409    1953542494
255208    1953388500
255878    1953540090
256

,orderid
7673,1955598428
10188,1954515646
17164,1953728724
32436,1956165008
39826,1954558468
...,...
452863,1955832452
453683,1955626470
462329,1953539378
466609,1953556460


In [31]:
fraud.drop_duplicates()

,orderid
7673,1955598428
10188,1954515646
17164,1953728724
32436,1956165008
39826,1954558468
...,...
884791,1955832452
886189,1955626470
902851,1953539378
910453,1953556460


# <center><h1> Test code </h1></center>

In [32]:
orders

,orderid,buyer_userid,seller_userid
0,1953278092,47388162,20822974
1,1953295120,26855196,16416890
2,1953306402,121296714,28477978
3,1953314712,131221930,72837912
4,1953381964,183398314,28423332
...,...,...,...
620942,1956636054,14222102,70940136
620943,1956642760,2978154,60766142
620944,1956651496,221974772,15541938
620945,1956660812,152533590,80986700


    * Lấy ra bảng chứa buyer_userid và thiết bị của họ

In [33]:
buyer_userid_device = orders.merge(devices, how='left', left_on='buyer_userid', right_on='userid')
dic = buyer_userid_device[['buyer_userid', 'device']]
dic = dic.rename(columns={'buyer_userid' : 'userid'})

In [34]:
dic = dic.rename(columns={'buyer_userid' : 'userid'})

In [35]:
dic

,userid,device
0,47388162,N6EVAPjl6WH2r1LOkkuwR0zKxT3fU8tubs3qvysuKTy=
1,47388162,NUDUHm6TKnxRN2Q+D7Wr2AAR5IyNabr1t3tobdD8uyq=
2,47388162,HJ5XUYvJpjF6xE1QlTAy0RQgXY1GZDskcoLSliOMNag=
3,47388162,5YKQkZDv4zi4RICd0AIVAS3Iv+awDbNxl49p51QF7fA=
4,47388162,HQgnByUSw54FHZwYDGgwCheMM72dnyBqqF7lDih1iMW=
...,...,...
1709743,54374608,s9tldgil9NLlyzVIv2BvkPQJIfUDGlTi5sYN6wy5640=
1709744,54374608,LQzl2WwY7aseGyS4rlgYDiWsfZowlF0LdHVSbJs/Am8=
1709745,54374608,mgvylksLNwhX/zwvYcm3Dlxf4KUEGW6VLasAgVWh8h8=
1709746,54374608,wBUFxlVwv3xaiM73yAywq9X2TV5DaC6+8YGgY2kJr4E=


    * Thực hiện lặp

    * Mức 1

In [36]:
new_device = buyer_userid_device

In [37]:
new = pd.merge(new_device, dic, how='left', left_on='device', right_on='device', suffixes=('_l', '_r')) #Nhập 2 bảng tìm thiết bị
new = new.dropna() #Đưa những orderid không có thiết bị ra ngoài
new

,orderid,buyer_userid,seller_userid,userid_l,device,userid_r
0,1953278092,47388162,20822974,47388162,N6EVAPjl6WH2r1LOkkuwR0zKxT3fU8tubs3qvysuKTy=,47388162
1,1953278092,47388162,20822974,47388162,NUDUHm6TKnxRN2Q+D7Wr2AAR5IyNabr1t3tobdD8uyq=,47388162
2,1953278092,47388162,20822974,47388162,HJ5XUYvJpjF6xE1QlTAy0RQgXY1GZDskcoLSliOMNag=,47388162
3,1953278092,47388162,20822974,47388162,5YKQkZDv4zi4RICd0AIVAS3Iv+awDbNxl49p51QF7fA=,47388162
4,1953278092,47388162,20822974,47388162,HQgnByUSw54FHZwYDGgwCheMM72dnyBqqF7lDih1iMW=,47388162
...,...,...,...,...,...,...
7244431,1956679514,54374608,10385548,54374608,wBUFxlVwv3xaiM73yAywq9X2TV5DaC6+8YGgY2kJr4E=,54374608
7244432,1956679514,54374608,10385548,54374608,wBUFxlVwv3xaiM73yAywq9X2TV5DaC6+8YGgY2kJr4E=,54374608
7244433,1956679514,54374608,10385548,54374608,bykv/AOAWbswGwihg0Jgy0AALEocpGE+G/j7rs1U0z4=,54374608
7244434,1956679514,54374608,10385548,54374608,bykv/AOAWbswGwihg0Jgy0AALEocpGE+G/j7rs1U0z4=,54374608


In [38]:
new = new[new['buyer_userid'] != new['userid_r']] #Loại bỏ những thằng bị trùng buyer_userid và userid_r (userid_r là tập những thằng có chung thiết bị với buyer)
new

,orderid,buyer_userid,seller_userid,userid_l,device,userid_r
50,1953480060,130236220,77145726,130236220,ef8eZZC3s2ktvq9157R8Sew08ST7c8KTbfm62bzEQ4u=,130437218
55,1953480060,130236220,77145726,130236220,xWG5A2p/3GE+JG6R3+EjRQK/5oKqhIhvbKFHr39+6UO=,130437218
60,1953480060,130236220,77145726,130236220,gO6bwYe5TsflvL9T4R2zP3JRgELYo5FmuDMcuAiW+cu=,130437218
81,1953537154,210510748,34245056,210510748,UP8gMqNw/V0HTeJRy+C0rkNM9KGKhOyC+JGNa82S96c=,233610178
82,1953537154,210510748,34245056,210510748,UP8gMqNw/V0HTeJRy+C0rkNM9KGKhOyC+JGNa82S96c=,233610178
...,...,...,...,...,...,...
7243708,1956221486,171658482,102234504,171658482,j8fnpI3h62/H5wt8jkFZ/B/4CfAjB7R21ghfpur579m=,145384558
7243807,1956221486,171658482,102234504,171658482,S98CiIqhfN7AfJqKr75J3R1Ju40e70t4Rk+FrxpC9qW=,145384558
7244398,1956636054,14222102,70940136,14222102,5euY3+8crcl9MLsTDtKJSjY3Sfj9P7YZiky48BE2g8m=,29044192
7244401,1956642760,2978154,60766142,2978154,27KLH9oMTCdU7BdSlESO/7bN0ExRHuo7yw2EuzpAlxq=,53187520


In [39]:
Fraud1 = new[new['seller_userid'] == new['userid_r']] #Tìm thằng gian lận khi seller_userid chính là userid_r (userid_r những thằng có chung thiết bị với buyer)
Fraud1

,orderid,buyer_userid,seller_userid,userid_l,device,userid_r
778689,1954135512,147864436,222975070,147864436,oSyoTN5PEg0uB6zm8VXTclZjpiWqTqVcSvg90wmU+TA=,222975070
778691,1954135512,147864436,222975070,147864436,Cuwr6pOA6fq0a4HXCi+MDNRtQBa4xnlQrr5LX6EOsmm=,222975070
778693,1954135512,147864436,222975070,147864436,etOkCx1lWrY49giVUb+SAhJYqaS2NRpru2cRIKsIJvq=,222975070
778695,1954135512,147864436,222975070,147864436,bMjPlcjjN7eAw0M75aAK1CyrnDyUUdyKAvwIRpFhs3u=,222975070
944754,1954926680,40831792,15853332,40831792,WquN8/hbZrrE401nmGTX3blxc5dQUEcvW9vWpeXeI+S=,15853332
944755,1954926680,40831792,15853332,40831792,WquN8/hbZrrE401nmGTX3blxc5dQUEcvW9vWpeXeI+S=,15853332
945116,1955597626,175581274,123961640,175581274,SgXy7CTjATxFacpXvmEzuDrBC/J6lAPWmn8tgVHNa4W=,123961640
1006969,1956353642,112909776,8044872,112909776,oXkO3X6XaHMKIiFNz+C6EfLUB9liRYVGx85GsCVGCpy=,8044872
1205048,1954696054,57355358,228510618,57355358,qZuBysx1/xPi6JhLD08h9KV+j0VwhCHCcbFt9K6Ih6S=,228510618
1945999,1955958696,223814202,124770776,223814202,2JYI7SmYrgsaCfVvYKVpp62e0Ac74NwzCq1EzazoMUm=,124770776


In [40]:
Fraud1.groupby('orderid').count().count()

buyer_userid     35
seller_userid    35
userid_l         35
device           35
userid_r         35
dtype: int64

In [41]:
new = new[new['seller_userid'] != new['userid_r']] #Loại bỏ những thằng đã phát hiện gian lận ở mức 1

In [42]:
new

,orderid,buyer_userid,seller_userid,userid_l,device,userid_r
50,1953480060,130236220,77145726,130236220,ef8eZZC3s2ktvq9157R8Sew08ST7c8KTbfm62bzEQ4u=,130437218
55,1953480060,130236220,77145726,130236220,xWG5A2p/3GE+JG6R3+EjRQK/5oKqhIhvbKFHr39+6UO=,130437218
60,1953480060,130236220,77145726,130236220,gO6bwYe5TsflvL9T4R2zP3JRgELYo5FmuDMcuAiW+cu=,130437218
81,1953537154,210510748,34245056,210510748,UP8gMqNw/V0HTeJRy+C0rkNM9KGKhOyC+JGNa82S96c=,233610178
82,1953537154,210510748,34245056,210510748,UP8gMqNw/V0HTeJRy+C0rkNM9KGKhOyC+JGNa82S96c=,233610178
...,...,...,...,...,...,...
7243708,1956221486,171658482,102234504,171658482,j8fnpI3h62/H5wt8jkFZ/B/4CfAjB7R21ghfpur579m=,145384558
7243807,1956221486,171658482,102234504,171658482,S98CiIqhfN7AfJqKr75J3R1Ju40e70t4Rk+FrxpC9qW=,145384558
7244398,1956636054,14222102,70940136,14222102,5euY3+8crcl9MLsTDtKJSjY3Sfj9P7YZiky48BE2g8m=,29044192
7244401,1956642760,2978154,60766142,2978154,27KLH9oMTCdU7BdSlESO/7bN0ExRHuo7yw2EuzpAlxq=,53187520


In [43]:
new = new.drop('device', axis=1) #Xóa cột device vì đã tìm thấy những user liên quan
new = new.drop('userid_l', axis=1) #Xóa cột userid_l vì userid_l chính là buyer_userid
new = new.drop_duplicates() #Xóa những hàng trùng nhau

In [44]:
new

,orderid,buyer_userid,seller_userid,userid_r
50,1953480060,130236220,77145726,130437218
81,1953537154,210510748,34245056,233610178
128,1953567064,158115678,103582530,232396836
190,1953702496,175905100,55708090,173149552
200,1953702496,175905100,55708090,28276504
...,...,...,...,...
7242752,1956044292,177299204,31421508,40632522
7242905,1956206048,58672028,39287026,22308064
7242967,1956221486,171658482,102234504,145384558
7244398,1956636054,14222102,70940136,29044192


    * Mức 2

In [45]:
new

,orderid,buyer_userid,seller_userid,userid_r
50,1953480060,130236220,77145726,130437218
81,1953537154,210510748,34245056,233610178
128,1953567064,158115678,103582530,232396836
190,1953702496,175905100,55708090,173149552
200,1953702496,175905100,55708090,28276504
...,...,...,...,...
7242752,1956044292,177299204,31421508,40632522
7242905,1956206048,58672028,39287026,22308064
7242967,1956221486,171658482,102234504,145384558
7244398,1956636054,14222102,70940136,29044192


In [46]:
device_new2 = pd.merge(new, dic, how='left', left_on='userid_r', right_on='userid', suffixes=('_l', '_r'))
device_new2

,orderid,buyer_userid,seller_userid,userid_r,userid,device
0,1953480060,130236220,77145726,130437218,130437218,gO6bwYe5TsflvL9T4R2zP3JRgELYo5FmuDMcuAiW+cu=
1,1953480060,130236220,77145726,130437218,130437218,ef8eZZC3s2ktvq9157R8Sew08ST7c8KTbfm62bzEQ4u=
2,1953480060,130236220,77145726,130437218,130437218,xWG5A2p/3GE+JG6R3+EjRQK/5oKqhIhvbKFHr39+6UO=
3,1953537154,210510748,34245056,233610178,233610178,UP8gMqNw/V0HTeJRy+C0rkNM9KGKhOyC+JGNa82S96c=
4,1953537154,210510748,34245056,233610178,233610178,lbO0l1WW38lZNSgRqlwdHnpdDhsgOL2uWw8QrsV9LUI=
...,...,...,...,...,...,...
544693,1956636054,14222102,70940136,29044192,29044192,SvVbHgVxap55BC5ZUUJnRtd5SbybAl2iaZaFGbv+QI8=
544694,1956642760,2978154,60766142,53187520,53187520,T4Vrh+qD46HjZ0usW0YqAnKFiMGrB8Mh6A0iqhmEcuc=
544695,1956642760,2978154,60766142,53187520,53187520,27KLH9oMTCdU7BdSlESO/7bN0ExRHuo7yw2EuzpAlxq=
544696,1956642760,2978154,60766142,53187520,53187520,aehCIdbgQYq83Z4S2MZesHpPgzUZOZybn2XJLigEh5u=


In [47]:
device_new2 = device_new2.dropna() #Xóa những thằng không có thiết bị
device_new2

,orderid,buyer_userid,seller_userid,userid_r,userid,device
0,1953480060,130236220,77145726,130437218,130437218,gO6bwYe5TsflvL9T4R2zP3JRgELYo5FmuDMcuAiW+cu=
1,1953480060,130236220,77145726,130437218,130437218,ef8eZZC3s2ktvq9157R8Sew08ST7c8KTbfm62bzEQ4u=
2,1953480060,130236220,77145726,130437218,130437218,xWG5A2p/3GE+JG6R3+EjRQK/5oKqhIhvbKFHr39+6UO=
3,1953537154,210510748,34245056,233610178,233610178,UP8gMqNw/V0HTeJRy+C0rkNM9KGKhOyC+JGNa82S96c=
4,1953537154,210510748,34245056,233610178,233610178,lbO0l1WW38lZNSgRqlwdHnpdDhsgOL2uWw8QrsV9LUI=
...,...,...,...,...,...,...
544693,1956636054,14222102,70940136,29044192,29044192,SvVbHgVxap55BC5ZUUJnRtd5SbybAl2iaZaFGbv+QI8=
544694,1956642760,2978154,60766142,53187520,53187520,T4Vrh+qD46HjZ0usW0YqAnKFiMGrB8Mh6A0iqhmEcuc=
544695,1956642760,2978154,60766142,53187520,53187520,27KLH9oMTCdU7BdSlESO/7bN0ExRHuo7yw2EuzpAlxq=
544696,1956642760,2978154,60766142,53187520,53187520,aehCIdbgQYq83Z4S2MZesHpPgzUZOZybn2XJLigEh5u=


In [48]:
new2 = pd.merge(device_new2, dic, how='left',left_on='device', right_on='device').dropna()
new2

,orderid,buyer_userid,seller_userid,userid_r,userid_x,device,userid_y
0,1953480060,130236220,77145726,130437218,130437218,gO6bwYe5TsflvL9T4R2zP3JRgELYo5FmuDMcuAiW+cu=,130236220
1,1953480060,130236220,77145726,130437218,130437218,gO6bwYe5TsflvL9T4R2zP3JRgELYo5FmuDMcuAiW+cu=,130437218
2,1953480060,130236220,77145726,130437218,130437218,gO6bwYe5TsflvL9T4R2zP3JRgELYo5FmuDMcuAiW+cu=,130236220
3,1953480060,130236220,77145726,130437218,130437218,gO6bwYe5TsflvL9T4R2zP3JRgELYo5FmuDMcuAiW+cu=,130236220
4,1953480060,130236220,77145726,130437218,130437218,gO6bwYe5TsflvL9T4R2zP3JRgELYo5FmuDMcuAiW+cu=,130236220
...,...,...,...,...,...,...,...
6892790,1956642760,2978154,60766142,53187520,53187520,T4Vrh+qD46HjZ0usW0YqAnKFiMGrB8Mh6A0iqhmEcuc=,2978154
6892791,1956642760,2978154,60766142,53187520,53187520,27KLH9oMTCdU7BdSlESO/7bN0ExRHuo7yw2EuzpAlxq=,53187520
6892792,1956642760,2978154,60766142,53187520,53187520,27KLH9oMTCdU7BdSlESO/7bN0ExRHuo7yw2EuzpAlxq=,2978154
6892793,1956642760,2978154,60766142,53187520,53187520,aehCIdbgQYq83Z4S2MZesHpPgzUZOZybn2XJLigEh5u=,53187520


In [49]:
new2 = new2.dropna() #Xóa nan
new2 = new2.drop('device', axis=1) # Xóa device vì đã tìm ra userid_y
new2 = new2.drop('userid_r', axis=1) # Xóa userid_r vì là trung gian
new2 = new2.drop('userid_x', axis=1) # Xóa userid_x vì là trung gian giống userid_r
new2 = new2[new2['buyer_userid'] != new2['userid_y']] #Loại bỏ những thằng buyer_userid là userid_y (vì chung id, chung tbi nen bo)
new2

,orderid,buyer_userid,seller_userid,userid_y
1,1953480060,130236220,77145726,130437218
6,1953480060,130236220,77145726,130437218
11,1953480060,130236220,77145726,130437218
16,1953537154,210510748,34245056,233610178
17,1953537154,210510748,34245056,233610178
...,...,...,...,...
6892788,1956636054,14222102,70940136,29044192
6892789,1956642760,2978154,60766142,53187520
6892791,1956642760,2978154,60766142,53187520
6892793,1956642760,2978154,60766142,53187520


In [50]:
new2 = new2.drop_duplicates() #Xóa duplicate
# new2 = new2.rename(columns={'userid' : 'userid_relate'})
new2

,orderid,buyer_userid,seller_userid,userid_y
1,1953480060,130236220,77145726,130437218
16,1953537154,210510748,34245056,233610178
56,1953567064,158115678,103582530,232396836
63,1953702496,175905100,55708090,173149552
78,1953702496,175905100,55708090,40943286
...,...,...,...,...
6892240,1956044292,177299204,31421508,40632522
6892332,1956206048,58672028,39287026,22308064
6892380,1956221486,171658482,102234504,145384558
6892783,1956636054,14222102,70940136,29044192


In [51]:
Fraud2 = new2[new2['seller_userid'] == new2['userid_y']]

In [52]:
Fraud2.groupby('orderid').count().count()

buyer_userid     34
seller_userid    34
userid_y         34
dtype: int64

    * Mức 3

Tiền xử lý mức 3

In [53]:
new2 = new2[new2['seller_userid'] != new2['userid_y']] #Loại bỏ những thằng đã gian lận ở mức 2
new2

,orderid,buyer_userid,seller_userid,userid_y
1,1953480060,130236220,77145726,130437218
16,1953537154,210510748,34245056,233610178
56,1953567064,158115678,103582530,232396836
63,1953702496,175905100,55708090,173149552
78,1953702496,175905100,55708090,40943286
...,...,...,...,...
6892240,1956044292,177299204,31421508,40632522
6892332,1956206048,58672028,39287026,22308064
6892380,1956221486,171658482,102234504,145384558
6892783,1956636054,14222102,70940136,29044192


In [54]:
device_new3 = pd.merge(new2, dic, how='left', left_on='userid_y', right_on='userid', suffixes=('_l, _r'))
device_new3

<ipython-input-54-831f67c22e64>:1: FutureWarning: Passing 'suffixes' as a <class 'str'>, is not supported and may give unexpected results. Provide 'suffixes' as a tuple instead. In the future a 'TypeError' will be raised.
  device_new3 = pd.merge(new2, dic, how='left', left_on='userid_y', right_on='userid', suffixes=('_l, _r'))


,orderid,buyer_userid,seller_userid,userid_y,userid,device
0,1953480060,130236220,77145726,130437218,130437218,gO6bwYe5TsflvL9T4R2zP3JRgELYo5FmuDMcuAiW+cu=
1,1953480060,130236220,77145726,130437218,130437218,ef8eZZC3s2ktvq9157R8Sew08ST7c8KTbfm62bzEQ4u=
2,1953480060,130236220,77145726,130437218,130437218,xWG5A2p/3GE+JG6R3+EjRQK/5oKqhIhvbKFHr39+6UO=
3,1953537154,210510748,34245056,233610178,233610178,UP8gMqNw/V0HTeJRy+C0rkNM9KGKhOyC+JGNa82S96c=
4,1953537154,210510748,34245056,233610178,233610178,lbO0l1WW38lZNSgRqlwdHnpdDhsgOL2uWw8QrsV9LUI=
...,...,...,...,...,...,...
585658,1956636054,14222102,70940136,29044192,29044192,SvVbHgVxap55BC5ZUUJnRtd5SbybAl2iaZaFGbv+QI8=
585659,1956642760,2978154,60766142,53187520,53187520,T4Vrh+qD46HjZ0usW0YqAnKFiMGrB8Mh6A0iqhmEcuc=
585660,1956642760,2978154,60766142,53187520,53187520,27KLH9oMTCdU7BdSlESO/7bN0ExRHuo7yw2EuzpAlxq=
585661,1956642760,2978154,60766142,53187520,53187520,aehCIdbgQYq83Z4S2MZesHpPgzUZOZybn2XJLigEh5u=


In [55]:
device_new3 = device_new3.dropna() #Xóa những thằng không có thiết bị
device_new3 = device_new3.drop('userid_y', axis=1) #Xóa trung gian
device_new3 = device_new3.drop('userid', axis=1) #Xóa trung gian
device_new3

,orderid,buyer_userid,seller_userid,device
0,1953480060,130236220,77145726,gO6bwYe5TsflvL9T4R2zP3JRgELYo5FmuDMcuAiW+cu=
1,1953480060,130236220,77145726,ef8eZZC3s2ktvq9157R8Sew08ST7c8KTbfm62bzEQ4u=
2,1953480060,130236220,77145726,xWG5A2p/3GE+JG6R3+EjRQK/5oKqhIhvbKFHr39+6UO=
3,1953537154,210510748,34245056,UP8gMqNw/V0HTeJRy+C0rkNM9KGKhOyC+JGNa82S96c=
4,1953537154,210510748,34245056,lbO0l1WW38lZNSgRqlwdHnpdDhsgOL2uWw8QrsV9LUI=
...,...,...,...,...
585658,1956636054,14222102,70940136,SvVbHgVxap55BC5ZUUJnRtd5SbybAl2iaZaFGbv+QI8=
585659,1956642760,2978154,60766142,T4Vrh+qD46HjZ0usW0YqAnKFiMGrB8Mh6A0iqhmEcuc=
585660,1956642760,2978154,60766142,27KLH9oMTCdU7BdSlESO/7bN0ExRHuo7yw2EuzpAlxq=
585661,1956642760,2978154,60766142,aehCIdbgQYq83Z4S2MZesHpPgzUZOZybn2XJLigEh5u=


In [56]:
new3 = pd.merge(device_new3, dic, how='left', left_on='device', right_on='device')
new3

,orderid,buyer_userid,seller_userid,device,userid
0,1953480060,130236220,77145726,gO6bwYe5TsflvL9T4R2zP3JRgELYo5FmuDMcuAiW+cu=,130236220
1,1953480060,130236220,77145726,gO6bwYe5TsflvL9T4R2zP3JRgELYo5FmuDMcuAiW+cu=,130437218
2,1953480060,130236220,77145726,gO6bwYe5TsflvL9T4R2zP3JRgELYo5FmuDMcuAiW+cu=,130236220
3,1953480060,130236220,77145726,gO6bwYe5TsflvL9T4R2zP3JRgELYo5FmuDMcuAiW+cu=,130236220
4,1953480060,130236220,77145726,gO6bwYe5TsflvL9T4R2zP3JRgELYo5FmuDMcuAiW+cu=,130236220
...,...,...,...,...,...
7231107,1956642760,2978154,60766142,T4Vrh+qD46HjZ0usW0YqAnKFiMGrB8Mh6A0iqhmEcuc=,2978154
7231108,1956642760,2978154,60766142,27KLH9oMTCdU7BdSlESO/7bN0ExRHuo7yw2EuzpAlxq=,53187520
7231109,1956642760,2978154,60766142,27KLH9oMTCdU7BdSlESO/7bN0ExRHuo7yw2EuzpAlxq=,2978154
7231110,1956642760,2978154,60766142,aehCIdbgQYq83Z4S2MZesHpPgzUZOZybn2XJLigEh5u=,53187520


In [57]:
new3 = new3.dropna() #Xóa nan
new3 = new3.drop('device', axis=1) # Xóa device vì đã tìm ra userid_y
new3 = new3[new3['buyer_userid'] != new3['userid']] #Xóa những buyer có trong userid
new3 = new3.drop_duplicates() #Xóa trùng lặp
new3

,orderid,buyer_userid,seller_userid,userid
1,1953480060,130236220,77145726,130437218
16,1953537154,210510748,34245056,233610178
56,1953567064,158115678,103582530,232396836
63,1953702496,175905100,55708090,173149552
78,1953702496,175905100,55708090,40943286
...,...,...,...,...
7230557,1956044292,177299204,31421508,40632522
7230649,1956206048,58672028,39287026,22308064
7230697,1956221486,171658482,102234504,145384558
7231100,1956636054,14222102,70940136,29044192


In [58]:
Fraud3 = new3[new3['seller_userid'] == new3['userid']]
Fraud3.groupby('orderid').count().count()

buyer_userid     34
seller_userid    34
userid           34
dtype: int64

In [59]:
# Fraud1['orderid'].drop_duplicates()

In [60]:
# Fraud2['orderid'].drop_duplicates()

In [61]:
# Fraud3['orderid'].drop_duplicates()